## FINANCIAL DATA

MODULE 3 | LESSON 1


---

# CALCULATING PORTFOLIO RISK/RETURN STATISTICS

|  |  |
|:---|:---|
|**Reading Time** |  40 minutes |
|**Prior Knowledge** | Simple Stock Returns, Variance, Python, Linear Algebra, Matrix Multiplication  |
|**Keywords** | Portfolio Return, Variance, Sharpe Ratio |

---

*The previous module had a focus on the individual security. For this lesson, we take what we have learned previously and apply it to a portfolio setting using a basket of stocks/crypto.*

In [1]:
import datetime
import math

import numpy as np
import pandas_datareader.data as web
from IPython.display import VimeoVideo

## 1. Portfolio Returns
Up to this point in the course, we've spent our time analyzing individual securities. Today, we perform a similar analysis except using a portfolio of assets. We will first quickly recap how to **determine return on investment for a single asset, General Electric (GE)**.

### 1.1 Single Asset Return Recap
We will assume we bought 100 shares 10 years ago. To determine the cash return, we just need to use the following formula:

$r_f = (p_f - p_i) * 100$

In [2]:
start = datetime.date(2011,11,29)
end = datetime.date(2021,11,28)
# start = datetime.date.today() - datetime.date.timedelta(365*10)
# end = datetime.date.today()

In [3]:
prices = web.DataReader(["GE"], "yahoo" , start, end, )["Adj Close"]

In [4]:
start = datetime.date.today() - datetime.timedelta(365*10)
end = datetime.date.today()
prices = web.DataReader(["GE"], "yahoo", start, end)["Adj Close"]
initialPrice = prices.GE[0]
finalPrice = prices.GE[-1]
cashReturn = (finalPrice - initialPrice)*100
print(
    f"With an initial investment of the ${np.round(initialPrice*100,2)}, the cash return of this investment would be ${np.round(finalPrice*100,2)} - ${np.round(initialPrice*100,2)} = ${np.round(cashReturn,2)}")

With an initial investment of the $12568.9, the cash return of this investment would be $6258.0 - $12568.9 = $-6310.9


### 1.2 How to Calculate Return with a Basket of Assets
We’ve now gone over returns of a single asset many times during this course. It’s time to apply similar logic to a basket of multiple assets. This will once again be best illustrated with an example. The following calculations can easily be extended to n number of securities, but to keep it simple, we will use two for our example. For this, we need to make some basic assumptions:

* 2 stocks
    * Meta—We will refer to it as its previous name from here on out: Facebook (FB) 
    * Chipotle (CMG)
* Bought 100 shares of each five years ago
* Goal: Calculate percentage return obtained by the portfolio at the end of the period

To start, we need to calculate the weights of each asset at the start of the period:

In [5]:
# Define all initial variables
# start = datetime.date.today()-datetime.timedelta(365*5)
# end = datetime.date.today()
#start = datetime.date(2016, 11, 29)
#end = datetime.date(2021, 11, 28)
prices = web.DataReader(["FB", "CMG"], "yahoo", start, end)["Adj Close"]
initialFB = prices.FB[0]
initialCMG = prices.CMG[0]
finalFB = prices.FB[-1]
finalCMG = prices.CMG[-1]
FBweight = initialFB / (initialFB+initialCMG)
CMGWeight =  initialCMG / (initialFB + initialCMG)

print(
    f"We have an initial investment in FB of ${np.round(initialFB * 100,2)} and in CMG ${np.round(initialCMG *100)}")

We have an initial investment in FB of $nan and in CMG $38037.0


In [6]:
print(
    f"This would make the weights {FBweight} and {CMGWeight} for FB and CMG respectively.")

This would make the weights nan and nan for FB and CMG respectively.


### 1.3 Final Returns

In order to calculate the final portfolio percentage returns, we need to find the returns of each asset individually, multiply the return by the weight in our portfolio, and then add them together. The formula to calculate this for two assets is:

$Portfolio_{Return} = w_1R_1 + w_2R_2$

In [7]:
returnFB = 100 * (finalFB - initialFB) / initialFB
returnCMG = 100 * (finalCMG - initialCMG) / initialCMG

In [8]:
print(f"This return over this periof for Facebook is {np.round(returnFB,2)} (FB) and {np.round(returnCMG,2)} (CMG)")

This return over this periof for Facebook is nan (FB) and 242.11 (CMG)


Adding these weighted returns together gives us a portfolio return of 41.14+250.35 = 291.49% 

Our portfolio would have returned 291.49% over the last five years, assuming we invested in both assets on the same starting date and with our weights.

In the first video of this lesson, we show how to transition from calculating an individual security's returns to calculating the returns of a portfolio of assets.

In [9]:
VimeoVideo("706655699", h="1a478ba2bf", width=600)

## 2. Calculating Portfolio Variance
This was discussed at a high level during the Financial Markets course, but here, we will show how to **calculate variance in Python with empirical data**. While returns are important, investors are also concerned about risk or volatility. We will use variance of returns to determine risk. Once again, we will use an example, using the same data as before to drive the point home. A bit of linear algebra is needed, but luckily, we have Python to do the tough calculations for us. 

If we were to calculate this for two assets using pencil and paper, we could use the following formula:

$\textrm{Portfolio variance} = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2Cov_{1,2}$

Where:

* $w_1$ = the portfolio weight of the first asset

* $w_2$ = the portfolio weight of the second asset

* $\sigma_1$ = the standard deviation of the first asset

* $\sigma_2$ = the standard deviation of the second asset

* $Cov_{1,2} = \textrm{the covariance of the two assets, which can thus be expressed as } \rho_{(1,2)}\sigma_1\sigma_2, \textrm{where }\rho_{(1,2)} \textrm{is the correlation coefficient between the two assets}$



Luckily, Python makes it easy for us to expand the formula to n number of assets. This formula is better expressed in matrix notation since it's easier to apply this way in Python: 

$Portfolio Variance = Weightstransposed * (covariance matrix) * weights$

### 2.1 Defining Variables
Let's put those portfolio weights we calculated before into an array and also calculate the simple daily returns of our asset along with a covariance matrix of these daily returns. Keep in mind that **will multiply the daily returns covariance matrix by 252 to measure annual variance.**



In [10]:
weights = np.array([0.23,0.77])
#print(weights)
returns = prices.pct_change()
# pct_change:
# Computes the percentage change from the immediately previous row by default. This is useful in comparing the percentage of change in a time series of elements.
covariance = 252* returns.cov()
print(covariance)

Symbols        FB       CMG
Symbols                    
FB       0.262849  0.101711
CMG      0.101711  0.127259


After defining these, we simply need to take the dot product of the transposed weights with the dot product of the covariance matrix and weights to get the annual variance of our portfolio. Note: **The t-method in numpy just transposes the array.**

In [11]:
variance = np.dot(weights.T, np.dot(covariance, weights))

# Print the result
print(str(np.round(variance, 4) * 100) + "%")

12.540000000000001%


We determine that the annual variance of our portfolio is 10.33%. Let’s compare this to the variance of each individual asset.

### 2.2 Comparing Portfolio Variance to Individual Stock Variance
This shows the annual variance of a portfolio with FB and CMG weighted 23% and 77% respectively has shown a 10.33% annual variance over the last five years.

One extremely interesting byproduct of this result can be seen when calculating variance of the equities individually. One of the main pillars of modern portfolio theory is about how diversification reduces risk. We can see this clear as day by looking at each stock's variance over the same time period.

In [12]:
returns.var() * 252

Symbols
FB     0.262849
CMG    0.127259
dtype: float64

We can see in above that both stocks' variance is higher than that of our portfolio, and even though CMG has a 77% weighting and a 13.7% variance, our portfolio's variance is more than 3 percentage points lower. This is because, if assets are not perfectly correlated, there will be some diversification benefit to risk by investing in multiple securities.

### 2.3 Calculating Standard Deviation of the Portfolio
Taking this a step further, since we know standard deviation is just the square root of variance, we can also calculate that here.

The below calculation shows the annual standard deviation of our portfolio to be about 32%:

In [13]:
np.round(math.sqrt(variance) * 100, 2)

35.41

Just like with the last video where we moved from thinking about individual stocks to a basket of stocks, we do the same here in order to calculate variance and standard deviation of returns for a portfolio.

In [14]:
VimeoVideo("706655745", h="8aa2f6d895", width=600)

## 3. Sharpe Ratio of Portfolio
Calculating the **Sharpe ratio of a portfolio** is pretty easy at this point since we have all the variables we need at our disposal. As a recap, the Sharpe ratio is just:

$\cfrac{Portfolio_{Return} - \textrm{risk-free rate}}{Portfolio_{StandardDev}}$

Since interest rates have been near 0 for a while now, we will assume 0 for the risk-free rate. Our formula is now: 

$\cfrac{\textrm{Portfolio}_{Return}}{\textrm{Portfolio}_{StandardDev}}$

We have calculated the total return of our portfolio over five years to be 291.49%. To get a single-year return, we can divide this number by 5 to get 58.298%. Considering our annual standard deviation is 32.01%, we can get the Sharpe ratio by dividing these two numbers to get 1.821 as the Sharpe ratio for our portfolio. In most cases, a Sharpe ratio over 1 is an exceptional risk/return metric. This makes sense considering how advantageous investments in Chipotle or Facebook would have been if you had invested in them five years ago.



## 4. Performing Similar Analysis on Cryptocurrencies
We will use the two most popular cryptocurrencies to perform a similar analysis, and we will compare and contrast them to the equity results we have obtained. We'll perform the analysis over the same time frame (the last five years) for the two most popular cryptocurrencies: Bitcoin and Ethereum. This time, let's say we bought five Bitcoin and 100 Ethereum five years ago.

In [15]:
# Define our initial variables
start = datetime.date.today() - datetime.timedelta(365*5)
end = datetime.date.today()
# start = datetime.date(2016,11,29)
# end = datetime.date(2021,11,28)
prices = web.DataReader(["BTC-USD", "ETH-USD"],"yahoo", start, end)["Adj Close"]
# <class 'pandas.core.frame.DataFrame'>
prices = prices.rename(columns = {"BTC-USD": "Bitcoin", "ETH-USD": "Ethereum"})
print(prices)
initialBTC = prices.Bitcoin[0] * 5
initialETH = prices.Ethereum[0] * 100
finalBTC = 10 * prices.Bitcoin[-1]
finalETH = 100 * prices.Ethereum[-1]
BTCWeight = initialBTC / (initialBTC + initialETH)
ETHWeight = initialETH / (initialBTC + initialETH)

Symbols          Bitcoin     Ethereum
Date                                 
2017-07-02   2506.469971          NaN
2017-07-03   2564.060059          NaN
2017-07-04   2601.639893          NaN
2017-07-05   2601.989990          NaN
2017-07-06   2608.560059          NaN
...                  ...          ...
2022-06-27  20735.478516  1193.680664
2022-06-28  20280.634766  1144.579224
2022-06-29  20104.023438  1098.943848
2022-06-30  19784.726562  1067.298828
2022-07-01  19330.812500  1057.579834

[1826 rows x 2 columns]


In [16]:
print(
    "This would make the weights %.2f and %.2f for Bitcoin and Ethereum respectively"
    % (BTCWeight, ETHWeight)
)

This would make the weights nan and nan for Bitcoin and Ethereum respectively


In [17]:
returnBTC = 100 * (finalBTC - initialBTC) / initialBTC
returnETH = 100 * (finalETH - initialETH) / initialETH

In [18]:
np.round(returnBTC, 3)

1442.473

In [19]:
np.round(returnETH, 3)

nan

Now, our returns for the stock portfolio were high, but this takes it to another level: Bitcoin and Ethereum have shown unprecedented returns over the last five years. Bitcoin returned 15,616.76% and Ethereum has returned 54,280.23%. If we keep using our portfolio weights from before, our portfolio would have returned 22,647.01%

In [20]:
np.round(returnBTC * BTCWeight + returnETH * ETHWeight, 3)

nan

### 4.1 Calculating Variance for Cryptos
Notice instead of multiplying by 252 like we did previously to annualize our variance, we use 365 here since cryptos trade seven days a week:

In [21]:
weights = np.array([0.82, 0.18])
returns = prices.pct_change()
covariance = 365 * returns.cov()
variance = np.dot(weights.T, np.dot(covariance, weights))
print(str(np.round(variance, 3) * 100) + "%")

63.3%


We get 63.6% variance for our portfolio, which translates to an annual standard deviation of 79.73%. Even though we were in relatively risky stocks, the standard deviation of that portfolio (32.14%) pales in comparison to this. Even though it's riskier, the crypto portfolio also returned a much higher rate. This illustrates a classic financial principle: With more risk, there is more potential return. Can you think of one simple way we can compare the portfolios taking into account risk AND return?

Yes, that's right, we can go back to the trusty Sharpe ratio.

### 4.2 Sharpe Ratio Comparison
The Sharpe ratio of our stock portfolio was 1.821. If we run the same calculation on our crypto portfolio we get:

22647.01/5 = 4,529.40% as our annual percentage return. If we divide this by the standard deviation we get:

45.294/.7973 = 56.81 Sharpe Ratio

This is a truly outstanding Sharpe ratio, and it's obvious at this point that if you had invested in Bitcoin or Ethereum five years ago and held it today, you're in very good shape. While this would've been a great investment, hindsight is 20/20, and it must be said that just because the last five years had incredible returns, it does not mean the next five years will.

## 5. Conclusion

During this lesson, we hint and imply that diversifying reduces risk in a portfolio. In the next lesson, we will prove it by quantifying the benefits of diversification.